In [1]:
###########################
# Author: Sarah Richard
# Case code: Python initiative
# Description: Intellisurvey crunching
# Last modified: 8/10/2020
###########################

In [2]:
import pandas as pd
import numpy as np
import re
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_string_dtype
from datetime import datetime

In [3]:
#helper functions
def answer_value_counts(data,qid):
    by_col=data.columns[qid]
    answerfreq=data.groupby(by_col)['Weight'].sum().sort_index().reset_index()
    answerfreq.rename(columns={by_col:'Answer'},inplace=True)
    if is_numeric_dtype(answerfreq['Answer']):
        answerfreq['Answer']=answerfreq['Answer'].astype(int).astype(str)
    else:
        answerfreq['Answer']=answerfreq['Answer'].astype(str)
    answerkey_qid=answerkey.loc[answerkey['Variable ID']==by_col,
                     ['Answer Code','Answer Label']]
    if is_numeric_dtype(answerkey_qid['Answer Code']):
        answerkey_qid['Answer Code']=answerkey_qid['Answer Code'].astype(int).astype(str)
    else:
        answerkey_qid['Answer Code']=answerkey_qid['Answer Code'].astype(str)
    answerfreq=answerfreq.merge(answerkey_qid,left_on='Answer',
                    right_on='Answer Code',how='right')
    answerfreq=answerfreq.iloc[:,[2,3,1]]
    answerfreq.rename(columns={answerfreq.columns[2]:'Counts'},inplace=True)
    answerfreq.fillna(0,inplace=True)
    
    if pd.api.types.is_string_dtype(answerfreq['Answer Code']):
        if answerfreq['Answer Code'].str.isdigit().all():
            answerfreq['Answer Code']=answerfreq['Answer Code'].astype(int)
    answerfreq.sort_values(by='Answer Code',inplace=True)
    total_N=pd.DataFrame({'Answer Code':['Total N'],'Counts':[data.loc[~data.iloc[:,qid].isna(),'Weight'].sum()]})
    answerfreq=answerfreq.append(total_N,sort=False,ignore_index=True)
    return answerfreq

def answer_rank_counts(data,qid):
    by_col=data.columns[qid]
    answerfreq=data.groupby(by_col)['Weight'].sum().sort_index().reset_index()
    answerfreq.rename(columns={by_col:'Answer'},inplace=True)
    if is_numeric_dtype(answerfreq['Answer']):
        answerfreq['Answer']=answerfreq['Answer'].astype(int).astype(str)
    else:
        answerfreq['Answer']=answerfreq['Answer'].astype(str)
    rank_key=pd.DataFrame({'Answer Code':['1','2','3'],'Answer Label':['1st','2nd','3rd']})
    answerfreq=answerfreq.merge(rank_key,left_on='Answer',
                    right_on='Answer Code',how='right')
    answerfreq=answerfreq.iloc[:,[2,3,1]]
    answerfreq.rename(columns={answerfreq.columns[2]:'Counts'},inplace=True)
    answerfreq.fillna(0,inplace=True)
    if pd.api.types.is_string_dtype(answerfreq['Answer Code']):
        if answerfreq['Answer Code'].str.isdigit().all():
            answerfreq['Answer Code']=answerfreq['Answer Code'].astype(int)
    answerfreq.sort_values(by='Answer Code',inplace=True)
    total_N=pd.DataFrame({'Answer Code':['Total N'],'Counts':["Check N separately"]})
    answerfreq=answerfreq.append(total_N,sort=False,ignore_index=True)
    return answerfreq

def answer_numericlist_counts(data,qid):
    data=data[~data[data.columns[qid]].isna()]
    if data.shape[0]>0:
        weighted_col=data.iloc[:,qid]*data['Weight']
        col_sort_by_value=data.loc[:,[data.columns[qid],'Weight']].sort_values(by=data.columns[qid])
        avg=weighted_col.sum()/data['Weight'].sum()
        col_sort_by_value['cumsum']=col_sort_by_value.Weight.cumsum()
        median_rank=col_sort_by_value['cumsum'].iloc[-1]/2
        median_rank_floor=np.floor(median_rank)
        median_rank_ceil=np.ceil(median_rank)
        col_sort_by_value['lower_than_median_floor']=col_sort_by_value['cumsum']<median_rank_floor
        col_sort_by_value['lower_than_median_ceil']=col_sort_by_value['cumsum']<median_rank_ceil
        median1=col_sort_by_value.loc[~col_sort_by_value['lower_than_median_floor'],data.columns[qid]].head(1).values[0]
        median2=col_sort_by_value.loc[~col_sort_by_value['lower_than_median_ceil'],data.columns[qid]].head(1).values[0]
        if median1==median2:
            median=median1
        else:
            median=median1+(median2-median1)/(median_rank_ceil-median_rank_floor)*(median_rank-median_rank_floor)
        col_sort_by_value['std']=np.power(col_sort_by_value[data.columns[qid]]-avg,2)*col_sort_by_value['Weight']
        std=np.sqrt(col_sort_by_value['std'].sum()/col_sort_by_value['Weight'].sum())
        total_count=data.loc[~data.iloc[:,qid].isna(),'Weight'].sum()
    else:
        avg='NA'
        median='NA'
        std='NA'
        total_count=0
    answerfreq=pd.DataFrame({'Answer Code':[1,2,3],'Answer Label':['Mean','Median','std'],
                            'Counts':[avg,median,std]})
    total_N=pd.DataFrame({'Answer Code':['Total N'],'Counts':[total_count]})
    answerfreq=answerfreq.append(total_N,sort=False,ignore_index=True)
    return answerfreq
            

In [4]:
time=datetime.now().strftime("Date_%m-%d_Time_%H-%M") # This is the time stamp. No need to edit this line

## <font color='red'>Input and output file names</font>

In [5]:
# Column names of the first question and last question in the survey raw datafile
# Case-sensitive
firstq='column name of the first question here'
# Make sure you enter the name of the **last column** of the last survey question if the question spans over multiple columns
# the last survey question is defined as the question **before** the closing questions "would you like to do an interview with us..."
lastq='column name of (the last column of) the last question here' 

# Intellisurvey sends raw data and datamap in the same excel file
# Here you only need to specify one file name (whereas for Owl there are two separate files for the raw data and datamap)
survey_raw_data_file='Data\[input file name].xlsx'
output_file='Analysis\[output file name]_'+time+'.xlsx'

# Load data and clean up format

In [6]:
# read survey data
survey=pd.read_excel(survey_raw_data_file,sheet_name=0)
answerkey=pd.read_excel(survey_raw_data_file,sheet_name=1,header=None)

# reformat datamap

answerkey=answerkey.rename(columns={0:"Answer Code",1:"Answer Label"})
answerkey=answerkey[~answerkey['Answer Code'].isna()]
qid_index=pd.Series(answerkey.index[answerkey['Answer Code']=='Type:'].tolist())-1
answerkey.loc[qid_index,'Variable ID']=answerkey.loc[qid_index,'Answer Code']
answerkey['Variable ID'].fillna(method='ffill',inplace=True)
answerkey.loc[qid_index,'Question']=answerkey.loc[qid_index,'Answer Label']
answerkey['Question'].fillna(method='ffill',inplace=True)
answerkey.loc[qid_index,'Type']=answerkey.loc[qid_index+1,'Answer Label'].tolist()
answerkey['Type'].fillna(method='ffill',inplace=True)
dtype_index=pd.Series(answerkey.index[answerkey['Answer Code']=='Datatype:'].tolist())
answerkey.loc[dtype_index-2,'Datatype']=answerkey.loc[dtype_index,'Answer Label'].tolist()
answerkey.loc[(~answerkey['Variable ID'].duplicated())& (answerkey['Type']=='text')&(answerkey['Datatype'].isna()),'Datatype']='text'
answerkey['Datatype'].fillna(method='ffill',inplace=True)
num_index=pd.Series(answerkey.index[(answerkey['Type']=='text')&(answerkey['Datatype']!='text')].tolist())
answerkey.loc[num_index,'Type']='numeric'

drop_index=pd.Series(answerkey.index[(answerkey['Answer Code']=='Type:') & (answerkey['Answer Label']!='text')& (answerkey['Answer Label']!='textbox')].tolist())-1
answerkey.drop(index=drop_index,inplace=True)
answerkey=answerkey[~answerkey['Answer Code'].isin(['Type:','Datatype:'])]
answerkey=answerkey.iloc[:,[3,2,4,0,1,5]]


answerkey['Question']=answerkey['Variable ID']+':'+answerkey['Question']
answerkey.loc[(answerkey['Type']=='checkbox')&(~answerkey['Variable ID'].str.contains('_DTA')),'Variable ID']=answerkey.loc[(answerkey['Type']=='checkbox')&(~answerkey['Variable ID'].str.contains('_DTA')),'Variable ID']+'.'+answerkey.loc[(answerkey['Type']=='checkbox')&(~answerkey['Variable ID'].str.contains('_DTA')),'Answer Code'].astype(str)
answerkey.reset_index(inplace=True)
start_index=(answerkey['Variable ID'].str.contains(firstq)).idxmax()
end_index=(answerkey['Variable ID'].str.contains(lastq)).idxmax()+1
answerkey=answerkey.iloc[start_index:end_index,:]

answerkey.loc[answerkey['Type']=='checkbox','Answer Code']='1'

### Loop through each question and count answers

In [7]:
df=pd.DataFrame({'Question ID':[''],'Question':[''],'Answer Code':[''],'Answer Label':[''],'Counts_all_completes':['']})
df2=pd.DataFrame({'Question ID':[''],'Question':[''],'Answer Code':[''],'Answer Label':['']})


## <font color='red'>Define weights, first question, single loop questions, and survey cuts here</font>

In [8]:
# DEFINE WEIGHTS HERE:
# Default weighting: 1
survey['Weight']=1 
# If you need to change weights from 1 to other values for a particular segment, define below
# EXAMPLE:
# survey.loc[survey['QFACTYPEQT']==2,'Weight']=2

# DEFINE COMPLETES HERE:
# Default cut: all completes
survey_complete=survey[(survey['status']=='C')]
# If you need to view incomplete responses, define below (may need to update depending on datamap)
# EXAMPLE:
# survey_complete=survey[(survey['status']=='I')]

# DO NOT EDIT SECTION BELOW...
qid_list=survey_complete.columns
questions=answerkey['Question'].unique()
startid=pd.Series(qid_list).str.contains(firstq).idxmax()
endid=pd.Series(qid_list).str.contains(lastq).idxmax()+1
# DO NOT EDIT SECTION ABOVE...

# ENTER SINGLE LOOP QUESTION #s HERE:
# If there are no single loop questions, write "loop_q=[]"
loop_q=['QXX','QYY..']
# Specify the indices and corresponding names of the looped items (i.e., vendors, brands, drugs)
# Note - ensure that the last looped option does NOT have a comma after
# EXAMPLE:
loop_dict={
#     1:'Brand 1',
#     2:'Brand 2',
#     ...
#     91:'Other'
}

# DEFINE ADDITIONAL CUTS HERE:
data_dict={
# Define survey cuts here
#     'cut_1':survey_complete[survey_complete['Q3']==1],
#     'cut_2':survey_complete[survey_complete['Q3']==2],
#     'cut_3':survey_complete[survey_complete['Q3']==3]
}

In [9]:
if len(loop_q)>0:
    loop_q_number=pd.Series()
    for loopq in loop_q:
        loop_q_number=loop_q_number.append(pd.Series(pd.Series(qid_list)[pd.Series(qid_list).str.startswith(loopq) & 
                                                                             ((pd.Series(qid_list).str.contains('R'))|(pd.Series(qid_list).str.contains('B')))]))
    loop_q_number=loop_q_number.reset_index(drop=True)    
    loop_q_number=pd.DataFrame(loop_q_number).rename(columns={0:'Variable ID'})
    loop_q_number[['Survey Question','index']]=pd.DataFrame(loop_q_number['Variable ID'].str.replace('_B','R').str.rsplit('R',n=1).tolist())
    loop_q_number.rename(columns={0:'Variable ID'},inplace=True)
    loop_list=pd.DataFrame.from_dict(loop_dict,orient='index',columns=['name']).reset_index()
    loop_list['index']=loop_list['index'].astype('str')
    loop_q_number=loop_q_number.merge(loop_list,on='index',how='left')
    loop_q_number.dropna(inplace=True)

In [10]:
# loop_q_number

In [11]:
for qid_index in list(range(startid,endid)):
    qid=survey_complete.columns[qid_index]
    if (len(loop_q)==0) or (qid not in loop_q_number["Variable ID"].values):
        df=df.append(pd.DataFrame({'Question ID':[qid],
                                   'Question':[answerkey.loc[answerkey['Variable ID']==qid,'Question'].iloc[0]],
                                   'Answer Code':[''],
                                   'Answer Label':[''],
                                   'Counts_all_completes':['']}),sort=False,ignore_index=True)

        if not(set(answerkey.loc[answerkey['Variable ID']==qid, 'Type']) & set(['numeric','ranking','textbox','pulldown'])): 

            answer=answer_value_counts(survey_complete,qid_index)
            answer.rename(columns={'Counts':'Counts_all_completes'},inplace=True)
            for data in data_dict.keys():
                cut=answer_value_counts(data_dict[data],qid_index)
                newname='Counts_'+data
                answer[newname]=cut['Counts']
            answer.reset_index(drop=True,inplace=True)
            df=pd.concat([df,answer],axis=0,sort=False)

        if answerkey.loc[answerkey['Variable ID']==qid, 'Type'].values[0]=='pulldown':

            answer=answer_rank_counts(survey_complete,qid_index)
            answer.rename(columns={'Counts':'Counts_all_completes'},inplace=True)
            for data in data_dict.keys():
                cut=answer_rank_counts(data_dict[data],qid_index)
                newname='Counts_'+data
                answer[newname]=cut['Counts']
            answer.reset_index(drop=True,inplace=True)
            df=pd.concat([df,answer],axis=0,sort=False)

        if answerkey.loc[answerkey['Variable ID']==qid, 'Type'].values[0]=='numeric':

            answer=answer_numericlist_counts(survey_complete,qid_index)
            answer.rename(columns={'Counts':'Counts_all_completes'},inplace=True)
            for data in data_dict.keys():
                cut=answer_numericlist_counts(data_dict[data],qid_index)
                newname='Counts_'+data
                answer[newname]=cut['Counts']
            answer.reset_index(drop=True,inplace=True)
            df=pd.concat([df,answer],axis=0,sort=False)


In [12]:
if len(loop_q)>0:
    for q in loop_q_number['Survey Question'].unique():
        q_label=loop_q_number.loc[loop_q_number['Survey Question']==q,'Variable ID'].reset_index(drop=True)
        qid=survey_complete.columns.get_loc(q_label[0])
        df2=df2.append(pd.DataFrame({'Question ID':[q],
                                     'Question':[answerkey.loc[answerkey['Variable ID']==q_label[0],'Question'].values[0]],
                                     'Answer Code':[''],
                                     'Answer Label':['']}),sort=False,ignore_index=True)
        name=loop_q_number.loc[loop_q_number['Variable ID']==q_label[0],'name'].values[0]
        if not(set(answerkey.loc[answerkey['Variable ID']==q_label[0], 'Type']) & set(['numeric','ranking','textbox','pulldown'])): 
            answer=answer_value_counts(survey_complete,qid)
            answer.rename(columns={'Counts':'Counts_all_completes_'+name},inplace=True)
            for ql in q_label[1:]:
                qid=survey_complete.columns.get_loc(ql)
                name=loop_q_number.loc[loop_q_number['Variable ID']==ql,'name'].values[0]
                cut=answer_value_counts(survey_complete,qid)
                newname='Counts_all_completes_'+name
                answer[newname]=cut['Counts']
            for data in data_dict.keys():
                for ql in q_label:
                    qid=survey_complete.columns.get_loc(ql)
                    name=loop_q_number.loc[loop_q_number['Variable ID']==ql,'name'].values[0]
                    cut=answer_value_counts(data_dict[data],qid)
                    newname='Counts_'+data+'_'+name
                    answer[newname]=cut['Counts']
            answer.reset_index(drop=True,inplace=True)
            df2=pd.concat([df2,answer],axis=0,sort=False)

        if answerkey.loc[answerkey['Variable ID']==q_label[0], 'Type'].values[0]=='pulldown':
            answer=answer_rank_counts(survey_complete,qid)
            answer.rename(columns={'Counts':'Counts_all_completes_'+name},inplace=True)
            for ql in q_label[1:]:
                qid=survey_complete.columns.get_loc(ql)
                name=loop_q_number.loc[loop_q_number['Variable ID']==ql,'name'].values[0]
                cut=answer_rank_counts(survey_complete,qid)
                newname='Counts_all_completes_'+name
                answer[newname]=cut['Counts']
            for data in data_dict.keys():
                for ql in q_label:
                    qid=survey_complete.columns.get_loc(ql)
                    name=loop_q_number.loc[loop_q_number['Variable ID']==ql,'name'].values[0]
                    cut=answer_rank_counts(data_dict[data],qid)
                    newname='Counts_'+data+'_'+name
                    answer[newname]=cut['Counts']
            answer.reset_index(drop=True,inplace=True)
            df2=pd.concat([df2,answer],axis=0,sort=False)

        if answerkey.loc[answerkey['Variable ID']==q_label[0], 'Type'].values[0]=='numericlist':
            answer=answer_numericlist_counts(survey_complete,qid)
            answer.rename(columns={'Counts':'Counts_all_completes_'+name},inplace=True)
            for ql in q_label[1:]:
                qid=survey_complete.columns.get_loc(ql)
                name=loop_q_number.loc[loop_q_number['Variable ID']==ql,'name'].values[0]
                cut=answer_numericlist_counts(survey_complete,qid)
                newname='Counts_all_completes_'+name
                answer[newname]=cut['Counts']
            for data in data_dict.keys():
                for ql in q_label:
                    qid=survey_complete.columns.get_loc(ql)
                    name=loop_q_number.loc[loop_q_number['Variable ID']==ql,'name'].values[0]
                    cut=answer_numericlist_counts(data_dict[data],qid)
                    newname='Counts_'+data+'_'+name
                    answer[newname]=cut['Counts']
            answer.reset_index(drop=True,inplace=True)
            df2=pd.concat([df2,answer],axis=0,sort=False)


In [13]:
writer = pd.ExcelWriter(output_file, engine='xlsxwriter')

# Write each dataframe to a different worksheet
df.to_excel(writer,sheet_name='Python output - nonloop',index=False)
df2.to_excel(writer,sheet_name='Python output - loop',index=False)

# Close the Pandas Excel writer and output the Excel file
writer.save()